# Decorators

In python, decorators act as functions that can take functions as input and give out functions as their output. Before we dive into decorators, there are few things we need to get out of the way

Functions can be assigned to variables :

In [3]:
def hello_world(name):
    return ("Hello world " + name)
    
print(hello_world("Jonathan"))

Hello world Jonathan


Functions can be defined within other functions :

In [7]:
def top_level_func(name):
    def bottom_level_func():
        return 'Using top level name in bottom level : ' + name
    return bottom_level_func()

print(top_level_func("Jonathan"))

# We can make this more explicit
def top_level_func(name):
    def bottom_level_func(first_name):
        return ('Using top level name in bottom level : ' + first_name)
    return bottom_level_func(name)

print(top_level_func("Jonathan"))

Using top level name in bottom level : Jonathan
Using top level name in bottom level : Jonathan


Functions can be passed to other functions :

In [8]:
def greeting(name):
    print ('Hello ' + name)
    
def top_func(greeting_func):
    # this function is used to pass a parameter to the greeing_func
    greeting_func("Snake wizard")
    
top_func(greeting)

Hello Snake wizard


Functions can return / generate new functions. Here the inner functions have access to the scope of the enclosure (enclosing scope).

In [13]:
def custom_greeting_generator(name):
    def greeting_template():
        return 'Hello, ' + name
    
    return greeting_template

custom_greeting_1 = custom_greeting_generator('Jonathan')
custom_greeting_2 = custom_greeting_generator('Sullivan')
print(custom_greeting_1())
print(custom_greeting_2())

Hello, Jonathan
Hello, Sullivan


How about a quadruple nested function ?

In [14]:
def top_level_func(username):
    def intermediate_level_func1():
        def intermediate_level_func2():
            def bottom_level_func():
                return 'Your username is ' + username
            return bottom_level_func()
        return intermediate_level_func2()
    return intermediate_level_func1()

print(top_level_func('Jonathan'))

Your username is Jonathan


Closures in python are defined as a technique/object that can be used to give a child function access to the its parent functions' variable. Here the child function is stored in the stack as a constant during its creation and the parent function's variable is kept alive until that function needs it, hence making it a closure.

In [15]:
def parent_func(name):
    def child_func():
        return 'The name of parent is ' + name
    return child_func

print_val = parent_func('Sinni')
print_val()

'The name of parent is Sinni'

Here's a counter example of a nested function that is not a closure. Here, it is not a closure because the parameter of the nested function has the value of the parent function's parameter bound (copied) to it and therefore it is initialized with this value beforehand and therefore the parent function's variable needn't stay alive.


In [16]:
def parent_func(name):
    def child_func(name=name):
        return 'The name of child is ' + name
    return child_func

print_val = parent_func('Sinni')
print_val()

'The name of child is Sinni'

Now when we create a decorator, the decorator is created keeping in mind that we know the signature of the function being passed to the decorator. This comes into play when we define the decorator, because we passed the arguments of the function being modified via the child of the decorator.

In [3]:
def hello_func(username):
    return 'Hello there, ' + username

def tag_decorator(func):
    def tag_wrapper(func_argument):
        return '<h1>' + func(func_argument) + '</h1>'
    # a decorator returns a function -> a new function that takes an argument and returns a html string as a response
    return tag_wrapper

decorated_func = tag_decorator(hello_func)
# now your decorated_func, i.e. your new function can take a string as input and give a HTML string as an output
# so, we could use this is as an interface for old and new functions / classes etc.
print(decorated_func('Remo'))

<h1>Hello there, Remo</h1>


Now, if we wanted to keep the name of our function the same, we could do the following :

In [8]:
def hello_func(username):
    return 'Hello there, fellow human - ' + username

print(hello_func('Arun'))

def decorator_func(func):
    def tag_wrapper(func_argument):
        return '<h1>' + func(func_argument) + '</h1>'
    return tag_wrapper

hello_func = decorator_func(hello_func)
# so that now calling our function takes the same input as before but gives a different output
print(hello_func('Arun'))

Hello there, fellow human - Arun
<h1>Hello there, fellow human - Arun</h1>


This can be written using some syntactic sugar aka Python's decorators

In [9]:
def html_decorator(func):
    def tag_wrapper(func_args):
        return '<h1>' + func_args + '</h1>'
    return tag_wrapper

# define function inline with the decorator
@html_decorator
def hello_user(username):
    return 'Hello, ' + username

print(hello_user('John'))

<h1>John</h1>


Decorators can also be chained ...

In [12]:
def htag_decorator(func):
    def tag_wrapper(func_args):
        return '<h1>' + func(func_args) + '</h1>'
    return tag_wrapper

def ptag_decorator(func):
    def tag_wrapper(func_args):
        return '<p>' + func(func_args) + '</p>'
    return tag_wrapper

def strong_tag_decorator(func):
    def tag_wrapper(func_args):
        return '<strong>' + func(func_args) + '</strong>'
    return tag_wrapper

@htag_decorator
@ptag_decorator
@strong_tag_decorator
def hello_user(username):
    return 'Hello, ' + username

print(hello_user('John'))

<h1><p><strong>Hello, John</strong></p></h1>


Decorators can be used with methods too, ...

In [15]:
def html_decorator(func):
    def tag_wrapper(func_args):
        return '<h1>' + func(func_args) + '</h1>'
    return tag_wrapper

class Person(object):
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    
    @html_decorator
    def get_name(self):
        return self.first_name + ' ' + self.last_name
    
person_1 = Person('Abcd', 'Efgh')
print(person_1.get_name())

<h1>Abcd Efgh</h1>


To pass arguments to our decorators, we will need to add in one additional level of complexity,

In [18]:
def tag_selector(tag_str):
    def html_decorator(func):
        def tag_wrapper(func_args):
            return '<{0}>{1}</{0}>'.format(tag_str, func(func_args))
        return tag_wrapper
    return html_decorator

@tag_selector('p')
def get_name(username):
    return 'Hello, ' + username

print(get_name('John'))
print(get_name.__name__)

<p>Hello, John</p>
tag_wrapper


We notice that the name of our function has changed and to fix this problem we can use the wrap decorator from functools ->

In [19]:
from functools import wraps

def tag_selector(tag_str):
    def html_decorator(func):
        @wraps(func)  # this decorator will set the name of our returned function back to the original name
        def tag_wrapper(func_args):
            return '<{0}>{1}</{0}>'.format(tag_str, func(func_args))
        return tag_wrapper
    return html_decorator

@tag_selector('p')
def get_name(username):
    return 'Hello, ' + username

print(get_name('John'))
print(get_name.__name__)

<p>Hello, John</p>
get_name
